In [1]:
import pandas as pd
import plotly.express as px
import cbsodata


In [2]:
# Fetch the specific dataset using its ID
data = pd.DataFrame(cbsodata.get_data('80072NED'))

In [4]:
# Extract year and quarter from the 'Perioden' column
data['Year'] = data['Perioden'].str.extract(r'(\d{4})').astype(float)
data['Quarter'] = data['Perioden'].str.extract(r'(\d)e kwartaal').astype(float)


In [5]:
# Drop rows where Year or Quarter is NaN (e.g., full year periods)
data = data.dropna(subset=['Year', 'Quarter'])

In [6]:
# Convert Year and Quarter to integers
data['Year'] = data['Year'].astype(int)
data['Quarter'] = data['Quarter'].astype(int)

In [7]:
# Filter data to include only years from 2016 to 2023
data = data[(data['Year'] >= 2016) & (data['Year'] <= 2023)]

In [8]:
# Aggregate the data to handle duplicates
agg_data = data.groupby(['Quarter', 'Year']).agg({'Ziekteverzuimpercentage_1': 'mean'}).reset_index()

In [9]:
# Pivot the table to get years as columns and quarters as rows
pivot_data = agg_data.pivot(index='Quarter', values='Ziekteverzuimpercentage_1', columns='Year')

In [14]:
# Melt the pivoted data for Plotly
melted_data = pivot_data.reset_index().melt(id_vars='Quarter', var_name='Year', value_name='Sick Leave Percentage')

In [28]:
# Custom Economist-style colors
economist_colors = [
    "#004b87",  # Dark Blue
    "#e3120b",  # Red
    "#767676",  # Gray
    "#bfbfbf",  # Light Gray
    "#6baed6",  # Light Blue
    "#fdae6b",  # Light Orange
    "#d62728",  # Crimson
    "#2ca02c",  # Green
]

# Create the Plotly line chart
fig = px.line(
    melted_data,
    x='Quarter',
    y='Sick Leave Percentage',
    color='Year',
    title='Sick Leave Across Quarters: Trends from 2016 to 2023',
    markers=True
)

# Update layout for a cleaner, minimalist style
fig.update_layout(
    template='none',  # No built-in theme
    title=dict(
        text='Sick Leave Across Quarters: Trends from 2016 to 2023',
        x=0.01, y=0.95,
        xanchor='left',
        yanchor='top',
        font=dict(family='Roboto', size=20, color='#004b87')
    ),
    xaxis=dict(
        title=None,
        tickmode='array',
        tickvals=[1, 2, 3, 4],
        ticktext=['Q1', 'Q2', 'Q3', 'Q4'],
        showgrid=True,
        gridcolor='#e5e5e5',
        linecolor='black',
        ticks='outside',
        mirror=False  # Remove top border
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#e5e5e5',
        linecolor='black',
        ticks='outside',
        mirror=False,  # Remove right border
        range=[2.5, agg_data['Ziekteverzuimpercentage_1'].max() + 1]
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=True,
    legend=dict(
        title=None,
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="right",
        x=1.1,
        font=dict(size=12, color='black', family='Roboto'),
        bordercolor='#e5e5e5',
        borderwidth=1
    ),
    width=900,
    height=600,
    font=dict(family='Roboto', size=12, color='black')
)

# Apply custom line and marker styles
fig.update_traces(
    line=dict(width=2),
    marker=dict(size=6, line=dict(width=0.5, color='black')),
    hovertemplate="<b>Year: %{customdata[0]}</b><br>Quarter: %{x}<br>Sick Leave: %{y:.2f}%<extra></extra>",
    customdata=melted_data[['Year']].values
)

# Use the custom Economist colors for lines
for i, trace in enumerate(fig.data):
    trace.update(line=dict(color=economist_colors[i % len(economist_colors)]))

# Add a subtitle
fig.add_annotation(
    text='Quarterly sick leave percentage by year (2016–2023)',
    xref='paper', yref='paper',
    x=0.01, y=1.1,
    showarrow=False,
    font=dict(family='Roboto', size=14, color='#767676'),
    xanchor='left',
    yanchor='top'
)

# Show the updated plot
fig.show()

# Save the plot as HTML for sharing
fig.write_html("economist_style_sickleave_percentage_updated.html")

c:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\uwv\.venv\Lib\site-packages\plotly\express\_core.py:1980: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

